In [1]:
import import_ipynb
from data603 import HDFS

httpdfs = HDFS.get_httpdfs()
httpdfs.list('/data/keras_models')

importing Jupyter notebook from /scratch/data603/yk98337/data603/HDFS.ipynb


['densenet',
 'efficientnet',
 'inception_resnet_v2',
 'inception_v3',
 'mobilenet',
 'mobilenet_v2',
 'mobilenet_v3',
 'nasnet',
 'resnet',
 'vgg16',
 'vgg19',
 'xception']

In [2]:
hdfs = HDFS.get_hdfs()

In [4]:
# Create a local directory
import os
keras_data = './keras_data'
if(not os.path.exists(keras_data)):
    os.mkdir(keras_data)

# download file from hdfs
mobilenet_weight_file = 'mobilenet_1_0_224_tf.h5'
local_weight_file = f"{keras_data}/{mobilenet_weight_file}"
if(not os.path.exists(local_weight_file)):
    httpdfs.download(f"/data/keras_models/mobilenet/{mobilenet_weight_file}", local_weight_file)

# check local file exists
os.listdir(keras_data)

['vgg16_weights_tf_dim_ordering_tf_kernels.h5',
 'resnet50_weights_tf_dim_ordering_tf_kernels.h5',
 'mobilenet_1_0_224_tf.h5']

In [5]:
import import_ipynb
from data603 import SparkLauncher

# get a configuration object
conf = SparkLauncher.get_spark_conf()

# add a file to the configuration that will get copied to all the nodes on the cluster
conf.set('spark.yarn.dist.files', './keras_data/mobilenet_1_0_224_tf.h5')

# launch the cluster using the configuration
spark = SparkLauncher.get_spark_session(pack_venv = False, conf = conf)

importing Jupyter notebook from /scratch/data603/yk98337/data603/SparkLauncher.ipynb
Creating Spark Configuration
Creating Spark Configuration
Setting Environment Variables
Creating Spark Session: yk98337_data603_spark_session


In [6]:
cats = spark.read.parquet('/user/yk98337/land_mammals.parquet')

In [7]:
cats = cats.filter("Label = 'Lion' OR Label = 'Tiger'")

In [39]:
def evaluate_chip(chip_data):
    import io
    import os
    import numpy as np
    from tensorflow.keras.applications.mobilenet import MobileNet
    from tensorflow.keras.applications.mobilenet import preprocess_input
    from tensorflow.keras.applications.mobilenet import decode_predictions
    from tensorflow.keras.preprocessing.image import load_img
    from tensorflow.keras.preprocessing.image import img_to_array

    # Load the image
    img = load_img(io.BytesIO(chip_data), target_size = (224,224))

    # Prepare Image
    image = img_to_array(img)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    # Load Model Data
    model = MobileNet(weights = f'{os.getcwd()}/mobilenet_1_0_224_tf.h5',
                 include_top = True,
                 alpha = 1.0)
    
    # Run prediction
    yhat = model.predict(image)

    # Decode Predictions
    features = np.argmax(yhat)

    
    return int(features)

In [40]:
# make a UDF
from pyspark.sql.types import *
from pyspark.sql.functions import udf

schema = IntegerType()

udf_evaluate_chip = udf(evaluate_chip, schema)


In [41]:
# get rid of the original image data
cats = cats.drop('Data')

# evaluate image chips
cats = cats.withColumn("features", udf_evaluate_chip("chip_data"))

In [42]:
cats5 = cats.select('ImageID','Label','features')

In [43]:
cats5.show(5)

+----------------+-----+--------+
|         ImageID|Label|features|
+----------------+-----+--------+
|00adbf6bf9f1848f| Lion|     245|
|0357cf336140cbe4|Tiger|     292|
|0401e49c378ffe99| Lion|     996|
|0bef987277f0189b|Tiger|     292|
|0e441053803b7dc5|Tiger|     292|
+----------------+-----+--------+
only showing top 5 rows



In [44]:
df = cats5.toPandas()
a = df[df['Label'] == Lion]['features']

In [46]:
import matplotlib.pyplot as plt
plt.scatter(df['features'])

TypeError: scatter() missing 1 required positional argument: 'y'

In [13]:
cats1 = cats.select('ImageID','Label','chip_data',cats.prediction[0],cats.prediction[1])

In [14]:
cats1 = cats1.withColumnRenamed('prediction[0]','predicted_label')
cats1 = cats1.withColumnRenamed('prediction[1]','predicted_score')

In [15]:
preds = cats1.select('predicted_label','Label').toPandas()

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 51714)
Traceback (most recent call last):
  File "/usr/lib64/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib64/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/scratch/data603_virtualenv/yk98337/lib64/python3.6/site-packages/pyspark/accumulators.py", line 267, in handle
    poll(authenticate_and_accum_updates)
  File "/scratch/data603_virtualenv/yk98337/lib64/python3.6/site-packages/pyspark/accumulators.py", line 239, in poll
    r, _, _ = select.select([self.rfile], [], [], 1)
ValueError: filedescriptor out of ran

In [16]:
preds['Label'] = preds['Label'].str.lower()

In [17]:
from sklearn.metrics import accuracy_score
accuracy_score(preds['predicted_label'],preds['Label'])

0.6515668631709587